In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
#Tensorflow log management: '1':info, '2':warning, '3': error

import time 

import tensorflow as tf
import tensorflow.contrib.layers as layers
from tensorflow.examples.tutorials.mnist import input_data
#Tutorials have mnist data files and queue

In [2]:
mnist = input_data.read_data_sets("/data/mnist", one_hot=True)
#MNIST images' has 28*28=764 features. Gray Image: Depth=1

Extracting /data/mnist/train-images-idx3-ubyte.gz
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
N_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 1

In [4]:
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder")
    Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder")

dropout = tf.placeholder(tf.float32, name='dropout') #Dropout Rate

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

In [5]:
with tf.variable_scope('conv1') as scope:
    # first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
    images = tf.reshape(X, shape=[-1, 28, 28, 1]) 
    kernel = tf.get_variable('kernel', [5, 5, 1, 32], 
                            initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [32],
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(images, kernel, strides=[1, 1, 1, 1], padding='SAME')
    conv1 = tf.nn.relu(conv + biases, name=scope.name)

    # output is of dimension BATCH_SIZE x 28 x 28 x 32
    conv1 = layers.conv2d(images, 32, 5, 1, activation_fn=tf.nn.relu, padding='SAME')

with tf.variable_scope('pool1') as scope:
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                           padding='SAME')

In [6]:
with tf.variable_scope('conv2') as scope:
    # similar to conv1, except kernel now is of the size 5 x 5 x 32 x 64
    kernel = tf.get_variable('kernels', [5, 5, 32, 64], 
                        initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [64],
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(pool1, kernel, strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv + biases, name=scope.name)

with tf.variable_scope('pool2') as scope:
    # similar to pool1
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='SAME')

In [7]:
with tf.variable_scope('fc') as scope:
    input_features = 7 * 7 * 64
    w = tf.get_variable('weights', [input_features, 1024],
                        initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [1024],
                        initializer=tf.constant_initializer(0.0))

    pool2 = layers.flatten(pool2)
    fc = tf.nn.relu(tf.matmul(pool2, w) + b, name='relu')
    # fc = layers.fully_connected(pool2, 1024, tf.nn.relu)

    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')

In [8]:
with tf.variable_scope('softmax_linear') as scope:
    w = tf.get_variable('weights', [1024, N_CLASSES],
                        initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [N_CLASSES],
                        initializer=tf.random_normal_initializer())
    logits = tf.matmul(fc, w) + b

In [9]:
with tf.name_scope('loss'):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    loss = tf.reduce_mean(entropy, name='loss')

In [10]:
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram_loss', loss)
    summary_op = tf.summary.merge_all()

In [11]:
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train = optimizer.minimize(loss, global_step=global_step)

In [12]:
os.mkdir('./checkpoints')
os.mkdir('./checkpoints/convnet_mnist')

In [13]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [14]:
# to visualize using TensorBoard
writer = tf.summary.FileWriter('./graphs/convnet', sess.graph)

In [15]:
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/convnet_mnist/checkpoint'))
# if that checkpoint exists, restore from checkpoint
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)

In [16]:
initial_step = sess.run(global_step)

start_time = time.time()
n_batches = int(mnist.train.num_examples / BATCH_SIZE)

total_loss = 0.0
for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
    X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
    _, loss_batch, summary = sess.run([train, loss, summary_op], 
                                      feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
    writer.add_summary(summary, global_step=index)
    total_loss += loss_batch
    if (index + 1) % SKIP_STEP == 0:
        print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
        total_loss = 0.0
        saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)

print("Optimization Finished!") # should be around 0.35 after 25 epochs
print("Total time: {0} seconds".format(time.time() - start_time))

Average loss at step 10: 2234.2
Average loss at step 20: 920.0
Average loss at step 30: 537.3
Average loss at step 40: 351.8
Average loss at step 50: 244.7
Average loss at step 60: 237.8
Average loss at step 70: 199.4
Average loss at step 80: 220.2
Average loss at step 90: 158.0
Average loss at step 100: 146.7
Average loss at step 110:  98.2
Average loss at step 120: 113.0
Average loss at step 130:  89.7
Average loss at step 140:  86.5
Average loss at step 150:  71.6
Average loss at step 160:  86.4
Average loss at step 170:  80.2


KeyboardInterrupt: 

In [ ]:
# test the model
n_batches = int(mnist.test.num_examples/BATCH_SIZE)
total_correct_preds = 0

for i in range(n_batches):
    X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
    _, loss_batch, logits_batch = sess.run([train, loss, logits], 
                                    feed_dict={X: X_batch, Y:Y_batch, dropout: 1.0})
    
    preds = tf.nn.softmax(logits_batch)
    correct_preds = tf.equal(tf.argmax(preds, axis=1), tf.argmax(Y_batch, axis=1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    total_correct_preds += sess.run(accuracy)   

total_test_num = int(mnist.test.num_examples/BATCH_SIZE)*BATCH_SIZE
print("Accuracy {0}".format(total_correct_preds/total_test_num))